### 📘 Lesson 9: SAPP Model

<div style="display: flex; align-items: center; justify-content: space-between;">
  <div>
    <h3>Course presenters</h3>
    <ul>
      <li><strong>Priyesh Gosai</strong> - Energy Systems Modeler and Training Coordinator</li>
      <li><strong>Dr. Ekaterina Fedotova</strong> - Senior Energy Systems Modeller</li>
      <li><strong>Albert Chitandula</strong> - Energy Systems Modeler</li>
    </ul>
  </div>
  <div>
    <a href="https://openenergytransition.org/index.html">
      <img src="https://openenergytransition.org/assets/img/oet-logo-red-n-subtitle.png" height="60" alt="OET">
    </a>
  </div>
</div>


#### 🎯 Learning Objectives  



* Build a model of the Southern African or East African Power Pools
---

**Setup Environment**

In [ ]:
# @title "Mount Google Drive and Set Working Directory"

from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Set FOLDER and change to the working directory in one step
FOLDER = 'ich-modeling-2025'
os.chdir(f'/content/drive/MyDrive/{FOLDER}')

# Confirm the current working directory
print("Current working directory:", os.getcwd())

In [ ]:
# @title Install latest version of PyPSA with Excel support

import os
from IPython.display import clear_output

# Install required packages
os.system("pip install -q pypsa")
os.system("pip install -q 'pypsa[excel]'")
os.system("pip install -q folium mapclassify")

# Clear the output after installation
clear_output()

# Notify the user
print("✅ All required packages have been installed.\n"
      "💡 To comment this cell, select all the text and press `Ctrl + /`.")

### 📄 **Case Study**

You have been provided

* The file `power_pool.xlsx` as a starting point to build a model. 
* The data provided in the folder `data/power_pool_data`
* Transfer Limits from the SAPP websites:
    * [SAPP](https://www.sapp.co.zw/transfer-limits)



#### 📈 Market Model

In [ ]:
import pypsa
import pandas as pd
import pathlib
pd.options.plotting.backend = 'matplotlib'

In [ ]:
path = pathlib.Path(pathlib.Path.cwd(), "data", "power_pool.xlsx")
network = pypsa.Network(path)


In [1]:
network.shapes

NameError: name 'network' is not defined

In [ ]:
network.plot.explore()

In [ ]:
network.snapshots

Shorten the simulation duration

In [ ]:

# start_date = pd.Timestamp("2027-08-06")  # change as needed
# simulation_duration = 30  # days, change as needed

# end_date = start_date + pd.Timedelta(days=simulation_duration)
# network.snapshots = network.snapshots[(network.snapshots >= start_date) & (network.snapshots < end_date)]
# print(f'Start: {network.snapshots[0]}') # Check first timestamp
# print(f'End: {network.snapshots[-1]}') # Check second timestamp
# print(f'Period: {network.snapshots[2] - network.snapshots[1]}') # Check period
# print(f'Timesteps: {network.snapshots.size}') # Number of timesteps

In [ ]:
network.generators

View Generators

In [ ]:
import matplotlib.pyplot as plt

# Exclude 'loadshedding', 'solar', and 'wind' generators
mask = ~network.generators.carrier.str.lower().isin(["loadshedding", "solar", "wind"])

# Attributes to plot
attributes = [
    ("p_nom", "Generator Capacity [MW]"),
    ("marginal_cost", "Marginal Price [currency/MWh]"),
    ("p_min_pu", "Minimum Power [pu]"),
    ("ramp_limit_up", "Ramp Limit Up/Down [pu]")
]

# 1 row, 4 columns, slightly narrower
fig, axes = plt.subplots(1, 4, figsize=(10, 8))

for i, (ax, (attr, title)) in enumerate(zip(axes, attributes)):
    data = network.generators.loc[mask, attr]
    data.plot(kind='barh', ax=ax)

    ax.set_title(title, fontsize=10)
    ax.tick_params(axis='x', labelsize=10)
    if i == 0:
        ax.tick_params(axis='y', labelsize=10)
    else:
        ax.set_yticklabels([])
        ax.tick_params(axis='y', length=0)  # hide ticks without touching font size

    
    if i > 0:
        ax.set_yticklabels([])
        ax.set_ylabel("")
    else:
        ax.set_ylabel("Generator", fontsize=10)

    ax.set_xlabel(attr, fontsize=12)

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Filter out 'loadshedding' carrier (case-insensitive)
gen_filtered = network.generators[~network.generators.carrier.str.lower().str.contains("loadshedding")]

# Group generator capacity by carrier
carrier_p_nom = gen_filtered.groupby('carrier')['p_nom'].sum()

# Plot as pie chart
fig, ax = plt.subplots(figsize=(3, 3))
carrier_p_nom.plot.pie(
    ax=ax,
    autopct='%1.1f%%',
    startangle=90,
    counterclock=False,
    ylabel='',              
    title='Generator Capacity Share by Carrier'
)

plt.tight_layout()
plt.show()

In [ ]:
import plotly.graph_objects as go

# Create a figure
fig = go.Figure()

# Add a trace for each generator
for gen in network.generators_t.p_max_pu.columns:
    fig.add_trace(go.Scatter(
        x=network.generators_t.p_max_pu.index,
        y=network.generators_t.p_max_pu[gen],
        mode='lines',
        name=gen
    ))

# Update layout
fig.update_layout(
    title="Maximum Power Output [pu]",
    xaxis_title="Time",
    yaxis_title="p_max_pu",
    template="plotly_white",
    height=600,
    width=1000
)

fig.show()


Storage unit data

In [ ]:
import matplotlib.pyplot as plt

# Attributes to plot for storage units
attributes = [
    ("p_nom", "Storage Power Capacity [MW]"),
    ("max_hours", "Storage Duration [h]"),
    ("efficiency_store", "Charging Efficiency"),
    ("efficiency_dispatch", "Discharging Efficiency")
]

fig, axes = plt.subplots(1, 4, figsize=(10, 3))

for i, (ax, (attr, title)) in enumerate(zip(axes, attributes)):
    data = network.storage_units[attr]
    data.plot(kind='barh', ax=ax)

    ax.set_title(title, fontsize=12)
    ax.set_xlabel(attr, fontsize=12)
    ax.tick_params(axis='x', labelsize=12)

    if i > 0:
        ax.set_yticklabels([])
        ax.set_ylabel("")
        ax.tick_params(axis='y', length=0)  # hide y-axis ticks
    else:
        ax.set_ylabel("Storage Unit", fontsize=12)
        ax.tick_params(axis='y', labelsize=12)

plt.tight_layout()
plt.show()


In [ ]:
import plotly.graph_objects as go

# Create Plotly figure
fig = go.Figure()

# Add a line for each load
for load in network.loads_t.p_set.columns:
    fig.add_trace(go.Scatter(
        x=network.loads_t.p_set.index,
        y=network.loads_t.p_set[load],
        mode='lines',
        name=load
    ))

# Update layout
fig.update_layout(
    title="Load Demand",
    xaxis_title="Time [h]",
    yaxis_title="Load Demand [MW]",
    template="plotly_white",
    height=600,
    width=1000
)

fig.show()


In [ ]:

network_node = network.copy()
# 1. Add a bus called 'South Africa' with carrier 'AC'
network_node.add("Bus", "South Africa", carrier="AC",x = 38.89780152023916, y = -77.03626158211897)


# 2. Remove all links and lines
network_node.links.drop(network_node.links.index, inplace=True)
network_node.lines.drop(network_node.lines.index, inplace=True)

# 3. Replace the bus for all generators and storage units to be 'South Africa'
network_node.generators['bus'] = 'South Africa'
network_node.storage_units['bus'] = 'South Africa'
network_node.loads['bus'] = 'South Africa'
# 4. Remove all other buses except 'South Africa'
buses_to_keep = ['South Africa']
buses_to_drop = network_node.buses.index.difference(buses_to_keep)
network_node.buses.drop(buses_to_drop, inplace=True)

In [ ]:
network.optimize()

In [ ]:
# network.optimize.optimize_with_rolling_horizon(horizon=24,overlap=0)

View results


In [ ]:
# Change plotting options to use the plotly toolbox for interactive plots
pd.options.plotting.backend = 'plotly'

In [ ]:
network.loads_t.p_set.plot()

In [ ]:
network.generators_t.p.plot()

In [ ]:
network.links_t.p0.plot()

Generator Dispatch

In [ ]:
import matplotlib.pyplot as plt

# Filter generator names containing 'loadshedding' (case-insensitive)
loadshedding_generators = [gen for gen in network.generators.index if 'loadshedding' in gen.lower()]


network.generators_t.p[loadshedding_generators].plot(title="Loadshedding Generators Output")



In [ ]:
# network.generators_t.p["Loadshedding"].sum()


In [ ]:
network.generators_t.p.describe()

In [ ]:
network.storage_units_t.state_of_charge.plot()

In [ ]:
network.storage_units_t.p.plot()

In [ ]:
import pandas as pd

# Assume your network is called 'network'
# 1. Merge generator power time series with generator attributes
gen_p = network.generators_t.p.copy()
gen_p['carrier'] = network.generators.carrier

# 2. Sum across time
total_energy_per_generator = gen_p.drop(columns="carrier").sum()

# 3. Map generators to carriers
carrier_map = network.generators.carrier
total_energy_per_carrier = total_energy_per_generator.groupby(carrier_map).sum()

print(total_energy_per_carrier)


In [ ]:
network.generators_t.p.plot()

In [ ]:
import plotly.graph_objects as go

# 1. Map generators to their carrier
carrier_map = network.generators.carrier

# 2. Group actual production per snapshot
power_per_carrier_per_snapshot = network.generators_t.p.T.groupby(carrier_map).sum().T


# 3. Calculate installed capacity for specific carriers
carrier_capacities = network.generators.groupby('carrier').p_nom.sum()

# 4. Start building the figure
fig = go.Figure()

# 5. Add traces for each carrier's production
for carrier in power_per_carrier_per_snapshot.columns:
    fig.add_trace(go.Scatter(
        x=power_per_carrier_per_snapshot.index,
        y=power_per_carrier_per_snapshot[carrier],
        mode='lines',
        name=carrier
    ))

# 6. Add horizontal lines for coal, nuclear, oil capacities
for selected_carrier in ['coal', 'nuclear', 'oil']:
    if selected_carrier in carrier_capacities.index:
        capacity = carrier_capacities[selected_carrier]
        fig.add_trace(go.Scatter(
            x=[power_per_carrier_per_snapshot.index.min(), power_per_carrier_per_snapshot.index.max()],
            y=[capacity, capacity],
            mode='lines',
            name=f"{selected_carrier} capacity ({capacity:.0f} MW)",
            line=dict(dash='dash')  # dashed line
        ))

# 7. Layout settings
fig.update_layout(
    title="Power Production by Carrier with Installed Capacities",
    xaxis_title="Time",
    yaxis_title="Power [MW]",
    legend_title="Carriers and Capacities",
    template="plotly_white",
    height=600
)

fig.show()

In [ ]:
network.generators_t.p.plot(kind="area")

In [ ]:
network.storage_units_t.p.plot(kind="area")

In [ ]:
# Copy the storage p dataframe
storage_p = network.storage_units_t.p.copy()

# Create two new DataFrames for dispatch and charge
storage_dispatch = storage_p.clip(lower=0)  # Keep only positive (discharge)
storage_charge = (storage_p.clip(upper=0))  # Flip sign for charging

# Rename the indices to distinguish
storage_dispatch.index.name = 'snapshot'
storage_dispatch.columns = [f"{col} dispatch" for col in storage_dispatch.columns]

storage_charge.index.name = 'snapshot'
storage_charge.columns = [f"{col} charge" for col in storage_charge.columns]

# Now, bring in the generator p
generator_p = network.generators_t.p.copy()
generator_p.index.name = 'snapshot'

# Concatenate everything
combined_df = pd.concat([generator_p, storage_dispatch, storage_charge], axis=1)

# Optional: sort columns if you want it tidy
combined_df = combined_df.sort_index(axis=1)



In [ ]:
combined_df.plot(kind="area", title="Power Production and Storage Dispatch/Charge")

In [ ]:
combined_df.columns

In [ ]:
# 1. Generators with "loadshedding" in the name (case-insensitive)
loadshedding_generators = [
    gen for gen in network.generators.index if "loadshedding" in gen.lower()
]

# 2. Storage dispatch names
storage_dispatch_names = [f"{name} dispatch" for name in network.storage_units.index]

# 3. Generators sorted by marginal cost, excluding loadshedding
non_loadshedding_generators = (
    network.generators[~network.generators.index.str.lower().str.contains("loadshedding")]
    .sort_values(by="marginal_cost", ascending=False)
    .index.tolist()
)

# 4. Storage charge names
storage_charge_names = [f"{name} charge" for name in network.storage_units.index]

# Combine all parts
plot_order = loadshedding_generators + storage_dispatch_names + non_loadshedding_generators + storage_charge_names

plot_order = plot_order[::-1]


In [ ]:
plot_order

In [ ]:
# Reorder the combined_df
# Drop missing ones first to avoid KeyError if any label doesn't exist
columns_in_df = [col for col in plot_order if col in combined_df.columns]

combined_df = combined_df[columns_in_df]


In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

for col in combined_df.columns:
    fig.add_trace(go.Scatter(
        x=combined_df.index,
        y=combined_df[col],
        mode='lines',
        stackgroup='one',
        name=col
    ))

fig.update_layout(
    title="Stacked Area Plot",
    xaxis_title="Time",
    yaxis_title="Power [MW]",
    legend_title="Units",
    # hovermode="x unified"
)

fig.show()


In [ ]:
network.storage_units_t.p.plot()

Marginal Price

In [ ]:


# Generator marginal costs
marginal_costs = network.generators.marginal_cost

# Generator dispatch [MW]
dispatch = network.generators_t.p

# Step 1: Compute market price at each timestep
def compute_market_price(dispatch_df, mc_series):
    prices = []
    for ts, row in dispatch_df.iterrows():
        dispatched = row[row > 0].index
        if dispatched.empty:
            prices.append(0)  # or np.nan
        else:
            prices.append(mc_series[dispatched].max())
    return pd.Series(prices, index=dispatch_df.index, name="market_price")


market_price = compute_market_price(dispatch, marginal_costs)

In [ ]:
market_price.plot()

Revenue

In [ ]:


# Step 2: Multiply dispatch by market price to get generator revenues at each timestep
revenue_per_generator_per_timestep = dispatch.mul(market_price, axis=0)

# Step 3: Sum over time to get total revenue per generator
total_revenue = revenue_per_generator_per_timestep.sum()

# Display or export
print(total_revenue.sort_values(ascending=False))
# total_revenue.to_csv("generator_revenue.csv")

###
---